# Fit Model


In [1]:
import pandas as pd
import mlutils
from sklearn import svm, metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# column names
feature_col = "features" # feature vector
value_col = "foldClass" # f# fold class to be predicted

### Read data set with fold type classifications and feature vectors

In [3]:
df = pd.read_json("./features.json")

In [4]:
print("Total number of data:", df.shape[0])
df.head(2)

Total number of data: 5370


,Exptl.,FreeRvalue,R-factor,alpha,beta,coil,features,foldType,length,ngram,pdbChainId,resolution,secondary_structure,sequence
1,XRAY,0.26,0.19,0.469945,0.046448,0.483607,"[0.1320128967, -0.4368766865, 0.65912170970000...",alpha,366,"[SR, RM, MP, PS, SP, PP, PM, MP, PV, VP, PP, P...",16VP.A,2.1,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...
1000,XRAY,0.23,0.18,0.504630,0.004630,0.490741,"[0.1890440972, -0.0811631067, 0.5008486872, 0....",alpha,216,"[ME, EA, AD, DV, VE, EQ, QQ, QA, AL, LT, TL, L...",1PBW.B,2.0,CCCCCCCCCCCCCCHHHHCCTTSCSCHHHHHHHHHHHHHHTTCTTT...,MEADVEQQALTLPDLAEQFAPPDIAPPLLIKLVEAIEKKGLECSTL...


## Split dataset into a training and test set

In [5]:
train, test = train_test_split(df, test_size=0.25, random_state=13, stratify=df[value_col])
print("Train set size:", train.shape[0])
print(train[value_col].value_counts())
print()
print("Test set size:", test.shape[0])
print(test[value_col].value_counts())
train.head(2)

KeyError: 'foldClass'

## Train a classifier

In [ ]:
classifier = svm.SVC(gamma='auto', class_weight='balanced', random_state=13)
#classifier = LogisticRegression(class_weight='balanced')
#classifier = RandomForestClassifier(random_state=13, class_weight='balanced')

classifier.fit(train[feature_col].tolist(), train[value_col].tolist())

## Make prediction for test set

In [ ]:
predicted = classifier.predict(test[feature_col].tolist())
expected = test[value_col].tolist()

## Calculate metrics for test set

In [ ]:
cm = metrics.confusion_matrix(expected, predicted)
print("Classification metrics:\n")
print(metrics.classification_report(expected, predicted))
mlutils.plot_confusion_matrix(cm, classifier.classes_, normalize=True, title='Normalized Confusion Matrix')
mlutils.plot_confusion_matrix(cm, classifier.classes_, normalize=False, title="Confusion Matrix")